<a href="https://colab.research.google.com/github/X-raiden/data-integration-4-scientific-dbs/blob/master/data_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
#imports
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import json
from scholarly import scholarly, ProxyGenerator
import requests

In [41]:
#scopus function
def scopus():
    data_details = []
    with open('web_of_science.json','r') as target:
        data = json.load(target)
    for i in range(0,6):
        resp = requests.get("http://api.elsevier.com/content/search/scopus?query=({"+data[i]['titre']+"})",headers=({"X-ELS-APIKey":"9d91974d46a4b958cafd78f34bbffcf0"}))
        #resp = requests.get("https://api.punkapi.com/v2/beers")
        data_details.append(resp.json())

    with open('scopus.json','w') as target:
        json.dump(data_details,target)
    return 1

In [42]:
#web of science function
def web_of_science(name_of_school):
    data = []
    #auteurs = []
    #keywords = []
    #it needs a geckodriver 'installed from github'
    driver = webdriver.Firefox(executable_path="/home/x-raiden/Downloads/geckodriver")
    driver.get("https://eressources.imist.ma/login?url=https://www.webofknowledge.com")
    #login to imist eressources
    login = driver.find_element_by_xpath("/html/body/div[1]/form/div[2]/input")
    password = driver.find_element_by_xpath("/html/body/div[1]/form/div[3]/input")
    #personal infos
    login.send_keys("ayoub.er-rkhis@usms.ac.ma")
    password.send_keys("@0000")
    button = driver.find_element_by_xpath("/html/body/div[1]/form/input")
    button.submit()
    #we should to wait for the page ( slow internet :'( )
    time.sleep(60)
    #entred to web_of_science
    main_selector = driver.find_element_by_xpath('//*[@id="select2-select1-container"]').click()
    search = driver.find_element_by_xpath('/html/body/span[37]/span/span[1]/input')
    search.send_keys("Enhanced")
    selector = driver.find_element_by_xpath('/html/body/span[37]/span/span[2]').click()
    search_bar = driver.find_element_by_xpath('//*[@id="value(input1)"]')
    search_bar.send_keys(name_of_school)
    button_in_web_of_science = driver.find_element_by_xpath("/html/body/form[1]/div[1]/div/div/div/table/tbody/tr/td[3]/span/span[1]/button")
    button_in_web_of_science.submit()
    #articles page
    time.sleep(30)
    driver.find_element_by_xpath('/html/body/div[1]/div[26]/div[2]/div/div/div/div[2]/div[3]/div[5]/form[2]/div/div[1]/div/span/div[2]/div[1]/div[3]/div/div[1]/div/a/value').click()
    time.sleep(16)
    for k in range (1,21):
        try:
            infos = {}
            infos['titre'] = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[1]/div/div[1]/value').text
            infos['date_publication'] = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[1]/div/div[3]/p[4]/value').text
            infos['type'] = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[1]/div/div[3]/p[5]').text
            infos['auteurs'] = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[1]/div/div[2]/p').text
            infos['auteur_mail'] = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[1]/div/div[6]/p[7]/a').text
            infos['abstract'] = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[1]/div/div[4]/p').text
            infos['keywords'] = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[1]/div/div[5]/p[1]').text
            infos['cited'] = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[2]/div[1]/div/div[6]/a/span').text
            infos['organization'] = 'Sultan Moulay Slimane University of Beni Mellal'
            data.append(infos)
            driver.find_element_by_xpath('/html/body/div[1]/div[26]/div/form/span/a[2]').click()
        except Exception as e:
            driver.find_element_by_xpath('/html/body/div[1]/div[26]/div/form/span/a[2]').click()
            pass

    with open('web_of_science.json','w') as target:
        json.dump(data,target)
    return 1


In [43]:
#google scholar function
def google_scholar():
    data_details = []
    #search_query = scholarly.search_pubs('Experimental and theoretical investigations of lignin-urea-formaldehyde wood adhesive: Density functional theory analysis')
    #scholarly.pprint(next(search_query))
    with open('web_of_science.json','r') as target:
        data = json.load(target)

    for i in range(0,6):
        name = data[i]['titre']
        search_query = scholarly.search_pubs(name)
        data_details.append(next(search_query)['bib'])
        #print(next(search_query))

    #print(data_details)
    with open('gs.json','w') as target:
        json.dump(data_details,target)
    return 1

In [65]:
#not yet completed!!!!!!!!!!!!!!!!
#science direct
def science_direct():
    data_details = []
    with open('web_of_science.json','r') as target:
        data = json.load(target)
    for i in range(0,6):
        resp = requests.get("https://api.elsevier.com/content/search/sciencedirect",headers=({"X-ELS-APIKey":"9d91974d46a4b958cafd78f34bbffcf0"}))
        #resp = requests.get("https://api.punkapi.com/v2/beers")
        #data_details.append(resp.json())
        print(resp.json())
    #with open('scopus.json','w') as target:
    #    json.dump(data_details,target)
    return 1

In [67]:
#main function
if __name__ == "__main__":
    name_of_school = "Sultan Moulay Slimane University of Beni Mellal "
    #web_of_science(name_of_school)
    #scopus()
    #google_scholar()
    science_direct()

{'service-error': {'status': {'statusCode': 'AUTHORIZATION_ERROR', 'statusText': 'The requestor is not authorized to access the requested view or fields of the resource'}}}
{'error-response': {'error-code': 'RATE_LIMIT_EXCEEDED', 'error-message': 'Rate of requests exceeds specified limits. Recommend lowering request rate and/or concurrency of requests.'}}
{'service-error': {'status': {'statusCode': 'AUTHORIZATION_ERROR', 'statusText': 'The requestor is not authorized to access the requested view or fields of the resource'}}}
{'error-response': {'error-code': 'RATE_LIMIT_EXCEEDED', 'error-message': 'Rate of requests exceeds specified limits. Recommend lowering request rate and/or concurrency of requests.'}}
{'service-error': {'status': {'statusCode': 'AUTHORIZATION_ERROR', 'statusText': 'The requestor is not authorized to access the requested view or fields of the resource'}}}
{'service-error': {'status': {'statusCode': 'AUTHORIZATION_ERROR', 'statusText': 'The requestor is not authoriz